In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [ ]:
folder = '/content/drive/MyDrive/Colab Notebooks/MagNet'

### Calling Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

### Load Data

In [ ]:
sunspots=pd.read_csv(os.path.join(folder, "sunspots.csv"), index_col=["period", "timedelta"],)
sat_pos = pd.read_csv(os.path.join(folder, "satellite_positions.csv"), index_col=["period", "timedelta"],)
dst = pd.read_csv(os.path.join(folder, "dst_labels.csv"), index_col=["period", "timedelta"],)
solar_wind = pd.read_csv(os.path.join(folder, "solar_wind.csv"),index_col=["period", "timedelta"],)

#### Function to summarize info

In [ ]:
def data_info(df):
  print ("shape of the data dst_labs:", df.shape)
  print ("Null values:", df.isnull().sum())
  print ("Total null values sanity check:", df.isnull().sum().sum())
  display(df.head())

#### Function to resample all dataframes
[Resample Help](https://www.w3resource.com/pandas/series/series-resample.php)  
Have to reset index

In [ ]:
def uri(df, sample_res):
    df = df.reset_index(level='period')
    df.index=pd.to_timedelta(df.index)
    return df.groupby('period').resample(sample_res).mean()

### Solar Wind interpolation and aggregation

In [ ]:
data_info(solar_wind)

shape of the data dst_labs: (8392320, 15)
Null values: bx_gse         325888
by_gse         325888
bz_gse         325888
theta_gse      325888
phi_gse        326388
bx_gsm         325888
by_gsm         325888
bz_gsm         325888
theta_gsm      325888
phi_gsm        326388
bt             325888
density        684890
speed          689555
temperature    811768
source         316816
dtype: int64
Total null values sanity check: 6088797


bx_gse  by_gse  bz_gse  ...   speed  temperature  source
period  timedelta                                ...                             
train_a 0 days 00:00:00   -5.55    3.00    1.25  ...  383.92     110237.0      ac
        0 days 00:01:00   -5.58    3.16    1.17  ...  381.79     123825.0      ac
        0 days 00:02:00   -5.15    3.66    0.85  ...  389.11      82548.0      ac
        0 days 00:03:00   -5.20    3.68    0.68  ...  389.11      82548.0      ac
        0 days 00:04:00   -5.12    3.68    0.49  ...  384.26      94269.0      ac

[5 rows x 15 columns]

##### Drop irrelevant columns from Solar Wind data

In [ ]:
# drop the unnecessary columns from solar_wind
cols_to_drop= ['bx_gsm','by_gsm', 'bz_gsm', 'theta_gsm', 'phi_gsm', 'source']
solar_wind.drop(cols_to_drop, axis=1, inplace=True)


In [ ]:
print ("The Null values in the data:\n", solar_wind.isnull().sum())

The Null values in the data:
 bx_gse         325888
by_gse         325888
bz_gse         325888
theta_gse      325888
phi_gse        326388
bt             325888
density        684890
speed          689555
temperature    811768
dtype: int64


In [ ]:
solar_wind.interpolate(method='linear', limit_direction='forward', axis=0, inplace=True)

In [ ]:
print ("shape of the data shape:", solar_wind.shape)
print ("Null values after interpolation:", solar_wind.isnull().sum().sum())
solar_wind.head()

shape of the data shape: (8392320, 9)
Null values after interpolation: 0


bx_gse  by_gse  bz_gse  ...  density   speed  temperature
period  timedelta                                ...                              
train_a 0 days 00:00:00   -5.55    3.00    1.25  ...     1.53  383.92     110237.0
        0 days 00:01:00   -5.58    3.16    1.17  ...     1.69  381.79     123825.0
        0 days 00:02:00   -5.15    3.66    0.85  ...     1.97  389.11      82548.0
        0 days 00:03:00   -5.20    3.68    0.68  ...     1.97  389.11      82548.0
        0 days 00:04:00   -5.12    3.68    0.49  ...     1.77  384.26      94269.0

[5 rows x 9 columns]

In [ ]:
sw_agg=uri(solar_wind, '30T')


In [ ]:
data_info(sw_agg)

shape of the data dst_labs: (279744, 9)
Null values: bx_gse         0
by_gse         0
bz_gse         0
theta_gse      0
phi_gse        0
bt             0
density        0
speed          0
temperature    0
dtype: int64
Total null values sanity check: 0


bx_gse    by_gse  ...       speed   temperature
period  timedelta                            ...                          
train_a 0 days 00:00:00 -5.924667  2.342667  ...  364.503167  79025.966667
        0 days 00:30:00 -6.374333  0.948333  ...  346.053333  58734.716667
        0 days 01:00:00 -6.470333  0.367333  ...  337.884667  78864.695238
        0 days 01:30:00 -6.917333  1.165667  ...  322.276500  25541.754762
        0 days 02:00:00 -7.215000  0.885333  ...  331.417833  55013.433333

[5 rows x 9 columns]

### Dst interpolation and join with Solar Wind data

In [ ]:
data_info(dst)

shape of the data dst_labs: (139872, 1)
Null values: dst    0
dtype: int64
Total null values sanity check: 0


dst
period  timedelta           
train_a 0 days 00:00:00   -7
        0 days 01:00:00  -10
        0 days 02:00:00  -10
        0 days 03:00:00   -6
        0 days 04:00:00   -2

In [ ]:
# sample only hourly data
dst= uri(dst, '30T')

In [ ]:
data_info(dst)

shape of the data dst_labs: (279741, 1)
Null values: dst    139869
dtype: int64
Total null values sanity check: 139869


dst
period  timedelta            
train_a 0 days 00:00:00  -7.0
        0 days 00:30:00   NaN
        0 days 01:00:00 -10.0
        0 days 01:30:00   NaN
        0 days 02:00:00 -10.0

In [ ]:
sw_dst=sw_agg.join(dst)
sw_dst.head()

bx_gse    by_gse  ...   temperature   dst
period  timedelta                            ...                    
train_a 0 days 00:00:00 -5.924667  2.342667  ...  79025.966667  -7.0
        0 days 00:30:00 -6.374333  0.948333  ...  58734.716667   NaN
        0 days 01:00:00 -6.470333  0.367333  ...  78864.695238 -10.0
        0 days 01:30:00 -6.917333  1.165667  ...  25541.754762   NaN
        0 days 02:00:00 -7.215000  0.885333  ...  55013.433333 -10.0

[5 rows x 10 columns]

In [ ]:
sw_dst['dst']=sw_dst[['dst']].interpolate(method='linear', limit_direction='forward', axis=0)

In [ ]:
data_info(sw_dst)

shape of the data dst_labs: (279744, 10)
Null values: bx_gse         0
by_gse         0
bz_gse         0
theta_gse      0
phi_gse        0
bt             0
density        0
speed          0
temperature    0
dst            0
dtype: int64
Total null values sanity check: 0


bx_gse    by_gse  ...   temperature   dst
period  timedelta                            ...                    
train_a 0 days 00:00:00 -5.924667  2.342667  ...  79025.966667  -7.0
        0 days 00:30:00 -6.374333  0.948333  ...  58734.716667  -8.5
        0 days 01:00:00 -6.470333  0.367333  ...  78864.695238 -10.0
        0 days 01:30:00 -6.917333  1.165667  ...  25541.754762 -10.0
        0 days 02:00:00 -7.215000  0.885333  ...  55013.433333 -10.0

[5 rows x 10 columns]

In [ ]:
print(sw_agg.shape , dst.shape, sw_dst.shape)

(279744, 9) (279741, 1) (279744, 10)


### Sunspots data interpolation and join with Solar Wind and Dst

In [ ]:
data_info(sunspots)

shape of the data dst_labs: (192, 1)
Null values: smoothed_ssn    0
dtype: int64
Total null values sanity check: 0


smoothed_ssn
period  timedelta                      
train_a 0 days 00:00:00            65.4
        13 days 00:00:00           72.0
        44 days 00:00:00           76.9
        74 days 00:00:00           80.8
        105 days 00:00:00          85.4

In [ ]:

sunspots = uri(sunspots, '30T')

In [ ]:
data_info(sunspots)

shape of the data dst_labs: (275283, 1)
Null values: smoothed_ssn    275091
dtype: int64
Total null values sanity check: 275091


smoothed_ssn
period  timedelta                    
train_a 0 days 00:00:00          65.4
        0 days 00:30:00           NaN
        0 days 01:00:00           NaN
        0 days 01:30:00           NaN
        0 days 02:00:00           NaN

In [ ]:
print(sw_agg.shape, dst.shape, sw_dst.shape, sunspots.shape)

(279744, 9) (279741, 1) (279744, 10) (275283, 1)


In [ ]:
sw_dst_ss= sw_dst.join(sunspots)
 # forward fill sunspot data for the rest of the month
sw_dst_ss['smoothed_ssn'] = sw_dst_ss['smoothed_ssn'].fillna(method="ffill")
# interpolate between missing solar wind values
sw_dst_ss['smoothed_ssn']=sw_dst_ss[['smoothed_ssn']].interpolate()

In [ ]:
print ("shape of all:", sw_agg.shape, dst.shape, sw_dst.shape, sunspots.shape)
print ("Total Null values:", sw_dst_ss.isnull().sum().sum())
display(sw_dst_ss.head())
display(sw_dst_ss.tail())

shape of all: (279744, 9) (279741, 1) (279744, 10) (275283, 1)
Total Null values: 0


bx_gse    by_gse  ...   dst  smoothed_ssn
period  timedelta                            ...                    
train_a 0 days 00:00:00 -5.924667  2.342667  ...  -7.0          65.4
        0 days 00:30:00 -6.374333  0.948333  ...  -8.5          65.4
        0 days 01:00:00 -6.470333  0.367333  ... -10.0          65.4
        0 days 01:30:00 -6.917333  1.165667  ... -10.0          65.4
        0 days 02:00:00 -7.215000  0.885333  ... -10.0          65.4

[5 rows x 11 columns]

bx_gse    by_gse  ...   dst  smoothed_ssn
period  timedelta                               ...                    
train_c 2435 days 21:30:00 -1.079667  3.882333  ... -14.5          42.5
        2435 days 22:00:00 -2.436667  3.454333  ... -15.0          42.5
        2435 days 22:30:00 -2.333333  2.266333  ... -14.5          42.5
        2435 days 23:00:00 -2.340667  2.987667  ... -14.0          42.5
        2435 days 23:30:00 -2.162333  2.251000  ... -14.0          42.5

[5 rows x 11 columns]

### Satellite Position Drop Cols and Interpolation

In [ ]:
data_info(sat_pos)

shape of the data dst_labs: (5828, 6)
Null values: gse_x_ace          0
gse_y_ace          0
gse_z_ace          0
gse_x_dscovr    4794
gse_y_dscovr    4794
gse_z_dscovr    4794
dtype: int64
Total null values sanity check: 14382


gse_x_ace  gse_y_ace  ...  gse_y_dscovr  gse_z_dscovr
period  timedelta                        ...                            
train_a 0 days     1522376.9   143704.6  ...           NaN           NaN
        1 days     1525410.9   136108.8  ...           NaN           NaN
        2 days     1528484.9   128470.5  ...           NaN           NaN
        3 days     1531570.3   120818.4  ...           NaN           NaN
        4 days     1534633.1   113178.5  ...           NaN           NaN

[5 rows x 6 columns]

In [ ]:
sat_pos = uri(sat_pos, '30T')

AttributeError: ignored

In [ ]:
# drop the unnecessary columns
cols_to_drop = ['gse_x_dscovr', 'gse_y_dscovr', 'gse_z_dscovr']
sat_pos.drop(cols_to_drop, axis=1, inplace=True)

print ('Droping the columns:', cols_to_drop)
print ('remaining columns:', sat_pos.columns )
sat_pos.head()

In [ ]:
df_final=df_sw_dst_ss.join(sat_pos)

In [ ]:
print ("Total Null values:", df_final.isnull().sum())

In [ ]:
cols_to_interp=['gse_x_ace', 'gse_y_ace', 'gse_z_ace']
df_final[cols_to_interp]=df_final[cols_to_interp].interpolate(method='linear', limit_direction='forward', axis=0)
print ('final_shape:', df_final.shape)
print ('Number of nulls:', df_sw_dst_ss.isnull().sum().sum() )
df_final.head(2)

In [ ]:
print ("shape of all:", sw_ipt_agg_ml.shape, dst.shape, df_sw_dst.shape, sat_pos.shape, df_final.shape)

In [ ]:
def agg_std(df_final, df_sw, sample_res):
    cols_to_keep = ['by_gse', 'bz_gse', 'bt', 'density', 'speed']
    df_sw = df_sw[cols_to_keep]
    
    df_sw = df_sw.reset_index(level='period')
    df_sw.index=pd.to_timedelta(df_sw.index)
    df_sw = df_sw.groupby('period').resample(sample_res).std()
    
    # rename the columns for the std
    columns = [c+'_std' for c in df_sw.columns]
    df_sw.columns = columns

    df = df_final.join(df_sw)
    return df

In [ ]:
df_final = agg_std(df_final, solar_wind, '30T')
print ('df_final.shape:', df_final.shape)
print ('Number of nulls:', df_final.isnull().sum().sum() )
df_final.head(2)

In [ ]:
df_to_save = df_final.reset_index(level='period')
df_to_save.head(10)

In [ ]:
df_to_save.tail(10)

In [ ]:
df_to_save.info()

In [ ]:
df_to_save.to_csv('/content/drive/MyDrive/Colab Notebooks/MagNet/df_30min.csv')